# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
!pip install -q torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q twine

Cannot remove entries from nonexistent file /opt/conda/lib/python3.6/site-packages/easy-install.pth
tensorflow-tensorboard 0.1.5 has requirement bleach==1.5.0, but you'll have bleach 3.3.1 which is incompatible.
moviepy 0.2.3.2 has requirement tqdm==4.11.2, but you'll have tqdm 4.61.2 which is incompatible.
awscli 1.16.17 has requirement colorama<=0.3.9,>=0.2.5, but you'll have colorama 0.4.4 which is incompatible.


In [32]:
# Imports here
import torch
from torchvision import datasets,transforms,models
from tqdm import tqdm
from torch import nn
from collections import OrderedDict
import random
from torch import optim
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import json
import torch.nn.functional as F
import pandas as pd
torch.__version__

'1.8.0+cu111'

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [33]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [34]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transform =transforms.Compose([transforms.RandomRotation(25),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_transform=transforms.Compose([transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor()])

validation_transform=transforms.Compose([transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor()])

# TODO: Load the datasets with ImageFolder
train_data =datasets.ImageFolder(train_dir,transform=train_transform)
test_data=datasets.ImageFolder(valid_dir,transform=test_transform)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloaders = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True)
testloaders=torch.utils.data.DataLoader(test_data,batch_size=64)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [35]:


with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [36]:
print(cat_to_name)
len(cat_to_name)
(cat_to_name)['68']

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

'bearded iris'

In [37]:
checkpoint = {'model_state_dict': model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict(),
             'epochs':20}

torch.save(checkpoint, 'checkpoint.pth')

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [38]:
# TODO: Build and train your network
model=models.resnet18(pretrained=True)

In [39]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [40]:
for name, child in model.named_children():
    print(name)

conv1
bn1
relu
maxpool
layer1
layer2
layer3
layer4
avgpool
fc


In [41]:


for name, child in model.named_children():
   if name in [ 'layer4','fc']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
            
    


classifier=nn.Sequential(OrderedDict([('lin1', nn.Linear(512,256)),
                                      ('relu1', nn.ReLU()),
                                      ('lin2', nn.Linear(256,102))
                                      ]))

model.fc=classifier

conv1 is frozen
bn1 is frozen
relu is frozen
maxpool is frozen
layer1 is frozen
layer2 is frozen
layer3 is frozen
layer4 is unfrozen
avgpool is frozen
fc is unfrozen


In [42]:
#print(model)

In [43]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device);

In [44]:
!nvidia-smi

Wed Jul 28 07:21:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    59W / 149W |   1766MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [45]:
test_data

Dataset ImageFolder
    Number of datapoints: 818
    Root Location: flowers/valid
    Transforms (if any): Compose(
                             Resize(size=255, interpolation=PIL.Image.BILINEAR)
                             CenterCrop(size=(224, 224))
                             ToTensor()
                         )
    Target Transforms (if any): None

In [46]:

indices =  random.sample(range(0, 8189), 2096)#list (range (0, 1048*2, 1))  # select your indices here as a list  
train_subset = torch.utils.data.Subset(train_data, indices)
test_subset = torch.utils.data.Subset(test_data, indices)

trainloaders_subset= torch.utils.data.DataLoader(train_subset,batch_size=32,shuffle=True)
testloaders_subset=torch.utils.data.DataLoader(test_subset,batch_size=64)

In [ ]:
# TODO: Do validation on the test set

optimizer=optim.SGD(model.parameters(),lr=0.003,momentum=0.9)
criterion=nn.CrossEntropyLoss()

epoch=20
step=0

train_losses,test_losses= [],[]
trainloaders1=trainloaders #_subset
testloaders1=testloaders #_subset

for e in range(epoch):
    running_loss=0
    for images,labels in tqdm(trainloaders1):
        # Move input and label tensors to the default device
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        running_loss=running_loss+loss.item()
    #train_losses.append(running_loss/len(trainloaders1))
    #print('train loss:{}'.format(train_losses))

        
    else:
        # TODO: Do validation on the test set
        test_loss=0
        accuracy=0
        
        with torch.no_grad():
            for images,labels in tqdm(testloaders1):
                # Move input and label tensors to the default device
                images,labels=images.to(device),labels.to(device)
                model.eval()
                output=model(images)
                loss=criterion(output,labels)
                
                ps=output #torch.exp(output)
                _,top_class=ps.topk(1,dim=1)
                equals=top_class==labels.view(*top_class.shape)
                accuracy=torch.mean(equals.type(torch.FloatTensor))
                
                test_loss=test_loss+loss.item()
                
            model.train()
            
            train_losses.append(running_loss/len(trainloaders1))
            test_losses.append(test_loss/len(testloaders1))
            
            print('train loss:{}'.format(train_losses))
            print('test loss:{}'.format(test_losses))
            print('accuracy:{}'.format(accuracy.item()*100))

100%|██████████| 13/13 [00:08<00:00,  1.51it/s]


train loss:[3.622649273058263]
test loss:[2.8504175589634824]
accuracy:28.00000011920929


100%|██████████| 13/13 [00:08<00:00,  1.53it/s]


train loss:[3.622649273058263, 1.602754096577807]
test loss:[2.8504175589634824, 1.735249427648691]
accuracy:63.999998569488525


100%|██████████| 13/13 [00:08<00:00,  1.51it/s]


train loss:[3.622649273058263, 1.602754096577807, 0.8772842106295795]
test loss:[2.8504175589634824, 1.735249427648691, 1.413725733757019]
accuracy:74.00000095367432


100%|██████████| 13/13 [00:08<00:00,  1.57it/s]


train loss:[3.622649273058263, 1.602754096577807, 0.8772842106295795, 0.6037822877488486]
test loss:[2.8504175589634824, 1.735249427648691, 1.413725733757019, 1.2943050173612742]
accuracy:66.00000262260437


 41%|████      | 84/205 [00:30<00:42,  2.82it/s]

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
model.class_to_idx = train_data.class_to_idx

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# TODO: Build and train your network
model=models.resnet18(pretrained=False)
classifier=nn.Sequential(OrderedDict([('lin1', nn.Linear(512,256)),
                                      ('relu1', nn.ReLU()),
                                      ('lin2', nn.Linear(256,102))]))

model.fc=classifier

In [ ]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model


model = load_checkpoint('checkpoint.pth')

In [ ]:
!ls checkpoint.pth

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
img_path='flowers/train/102/image_08000.jpg'
img=Image.open(img_path)
set_dim=256
if img.size[0] <= img.size[1]:
    factor=float(img.size[0])
    percent = (set_dim/(factor))
    hsize = int((float(img.size[1])*float(percent)))
    img = img.resize((set_dim,hsize), Image.ANTIALIAS)
else:
    factor=float(img.size[1])
    percent = (set_dim/(factor))
    wsize = int((float(img.size[0])*float(percent)))
    img = img.resize((wsize,set_dim), Image.ANTIALIAS)
    
new_width=224
new_height=224

width, height = img.size   # Get dimensions

left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2

# Crop the center of the image
img = img.crop((left, top, right, bottom))

np_image = np.array(img)/255
#img1 = Image.fromarray(data, 'RGB')


mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
for i in range(np_image.shape[2]):
    np_image[i] = (np_image[i] - mean[i]) / std[i]


a=torch.from_numpy(np_image.transpose(2,0,1).astype(np.float32))

plt.imshow(transforms.ToPILImage()(a), interpolation="bicubic")



In [ ]:
def process_image(img_path):
    
    img=Image.open(img_path)
    set_dim=256
    if img.size[0] <= img.size[1]:
        factor=float(img.size[0])
        percent = (set_dim/(factor))
        hsize = int((float(img.size[1])*float(percent)))
        img = img.resize((set_dim,hsize), Image.ANTIALIAS)
    else:
        factor=float(img.size[1])
        percent = (set_dim/(factor))
        wsize = int((float(img.size[0])*float(percent)))
        img = img.resize((wsize,set_dim), Image.ANTIALIAS)

    new_width=224
    new_height=224

    width, height = img.size   # Get dimensions

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    img = img.crop((left, top, right, bottom))

    np_image = np.array(img)/255

    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    for i in range(np_image.shape[2]):
        np_image[i] = (np_image[i] - mean[i]) / std[i]


    return  torch.from_numpy(np_image.transpose(2,0,1).astype(np.float32))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
img_path=('flowers/test/1/image_06752.jpg')
input=process_image(img_path)
plt.imshow(transforms.ToPILImage()(input), interpolation="bicubic")

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    input=process_image(image_path)
    input.unsqueeze_(0)
    with torch.no_grad():
        model.eval()
        output=model(input)

    output.shape
    probability=output
    _,top_class=probability.topk(5,dim=1)
    return torch.index_select(F.softmax(output,dim=1).squeeze(),0,top_class.squeeze()).tolist(),top_class.squeeze().tolist()


In [ ]:
probs, classes = predict(img_path, model)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classesinput=process_image(img_path)

img_path='flowers/train/102/image_08000.jpg'
probs, classes = predict(img_path, model)

prediction=[cat_to_name[x] for x in [str(x) for x in classes]]
df=pd.DataFrame(list(zip(prediction, probs)),columns =['prediction', 'probs'])

In [ ]:
plt.imshow(transforms.ToPILImage()(process_image(img_path)), interpolation="bicubic")
ax = df.plot.barh(x='prediction', y='probs', rot=0)